In [31]:
import os
import pandas as pd
# pd.set_option('display.max_columns', None)
# xls 파일이 있는 폴더 경로
folder_path = 'C:/Users/cj033/work/forest_fire/visitor'

def filter_sum(df):
    return df[df['내/외국인'] == '합계']

# 폴더 내 모든 xls 파일 가져오기
xls_files = [f for f in os.listdir(folder_path) if f.endswith('.xls')]

# 불러온 파일명을 저장할 리스트
loaded_files = []

# 파일마다 읽어서 변수로 저장하고 필터링 적용
for file in xls_files:
    # 파일 경로 만들기
    file_path = os.path.join(folder_path, file)
    
    # 파일을 읽어오기
    df = pd.read_excel(file_path)
    
    # 파일명을 변수명으로 설정하기 (확장자 제외)
    var_name = os.path.splitext(file)[0]
    
    # 변수에 DataFrame 할당
    globals()[var_name] = df
    
    # 필터링 적용
    globals()[var_name + '_sum'] = filter_sum(df)
    
    # 불러온 파일명 리스트에 추가
    loaded_files.append(var_name+'_sum')

# 모든 데이터프레임을 하나로 합치기
all_data = pd.concat([globals()[name] for name in loaded_files], ignore_index=True)
all_data.to_csv('visitor.csv', index=False)


In [33]:
all_data = pd.read_csv('visitor.csv')

In [37]:
import pandas as pd
from sqlalchemy import create_engine

# PostgreSQL 연결 설정
DB_USER = "postgres"  # 사용자 이름
DB_PASSWORD = "8350"  # 비밀번호
DB_HOST = "localhost"  # 데이터베이스 호스트 (예: localhost)
DB_PORT = "5432"  # 포트 (기본값: 5432)
DB_NAME = "postgres"  # 데이터베이스 이름

# SQLAlchemy 엔진 생성
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# CSV 또는 기존 DataFrame 로드 (여기서는 예시로 DataFrame 사용)
df = pd.read_csv("visitor.csv")  # 또는 df = your_dataframe

# PostgreSQL 테이블에 데이터 삽입
TABLE_NAME = "tourism_data"

# 데이터프레임을 SQL 테이블로 저장 (테이블이 없으면 생성, 존재하면 데이터 추가)
df.to_sql(TABLE_NAME, con=engine, if_exists="append", index=False)

print("데이터 삽입 완료!")


데이터 삽입 완료!


In [ ]:
# process_dataframe 함수 정의
def process_dataframe(df):
    df.fillna(0, inplace=True)
    df_grouped = df.groupby('시도').sum()
    df_grouped = df_grouped.loc[:, ~df_grouped.columns.str.contains('인원계')]
    df_grouped.drop(columns=['군구', '관광지', '내/외국인', '총계(2016.01 ~ 2024.12)'], inplace=True)
    df_grouped.columns = [col.replace('년 ', '-').replace('월', '') for col in df_grouped.columns[:]]
    df_grouped = df_grouped.reset_index()
    return df_grouped

all_data_processed = process_dataframe(all_data)

# 변경할 매핑 딕셔너리
mapping = {
    '강원특별자치도': '강원',
    '경기도': '경기',
    '경상남도': '경남',
    '경상북도': '경북',
    '광주광역시': '광주',
    '대전광역시': '대전',
    '울산광역시': '울산',
    '인천광역시': '인천',
    '전라남도': '전남',
    '전북특별자치도': '전북',
    '제주특별자치도': '제주',
    '충청남도': '충남',
    '충청북도': '충북'
}

# 값 변경 적용
all_data_processed['시도'] = all_data_processed['시도'].replace(mapping)

In [41]:
all_data_processed.head()

,시도,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,강원,104181.0,66931.0,27779.0,36805.0,118456.0,102667.0,110446.0,155035.0,82898.0,...,55902.0,0.0,0.0,0.0,5611.0,8186.0,4589.0,14756.0,7644.0,1449.0
1,경기,194351.0,172562.0,244989.0,306380.0,339828.0,316205.0,390414.0,211075.0,331222.0,...,65663.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,경남,85434.0,98319.0,99401.0,133290.0,159593.0,128100.0,168601.0,229442.0,88989.0,...,106230.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,경북,223751.0,217081.0,228890.0,273544.0,329993.0,341288.0,351099.0,357547.0,248977.0,...,184218.0,270222.0,306466.0,203520.0,158015.0,232300.0,194703.0,368237.0,307310.0,150616.0
4,광주,302941.0,270635.0,258158.0,222812.0,253583.0,215584.0,194079.0,228361.0,207768.0,...,155962.0,212406.0,214401.0,161664.0,116318.0,128112.0,143580.0,88222.0,86884.0,62905.0
